In [1]:
'''Main'''
import numpy as np
import pandas as pd
import os

'''Data Viz'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''Data Prep'''
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.compose import ColumnTransformer 
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import precision_recall_curve, average_precision_score,precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report 

'''Algos'''
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import autogluon.core as ag
from autogluon import TabularPrediction as task
#import xgboost as xgb
#import lightgbm as lgb

# Data Preparation

## Acquire Data

In [2]:
# We only get the train data set
current_path = os.getcwd()+"\\"
file = os.path.sep.join(["..\\data\\train.csv"])
data = pd.read_csv(current_path + file)
data=data.set_index("Id")

# signal is the target variable
df=data.drop(["BUTTER"], axis=1) # Droping BUTTER variable as is IRRELEVANT

In [3]:
msk = np.random.rand(len(df)) < 0.99

train = df[msk]

test = df[~msk]

# AutoGluon Inital

In [52]:
train_data = task.Dataset(train)
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)

In [60]:
label_column = 'signal'
print("Summary of class variable: \n", train_data[label_column].describe())

Summary of class variable: 
 count    210568.000000
mean          0.334001
std           0.471641
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: signal, dtype: float64


In [30]:
dir = current_path+"\\trained_models"
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Beginning AutoGluon training ...
AutoGluon will save models to C:\Users\garciagr\OneDrive - HP Inc\DSUB\MachineLearning\Boson\2020.ml.kaggle\Marc\\trained_models\
AutoGluon Version:  0.0.14
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    39187.19 MB
	Train Data (Original)  Memory Usage: 0.06 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


In [32]:
test_data = task.Dataset(test)
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(labels=[label_column],axis=1)  # delete label column to prove we're not cheating

In [39]:
predictor = task.load(dir)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.7050730063646574
Evaluations on test data:
{
    "accuracy": 0.7050730063646574,
    "accuracy_score": 0.7050730063646574,
    "balanced_accuracy_score": 0.5768322523659386,
    "matthews_corrcoef": 0.2510302944295476,
    "f1_score": 0.7050730063646574
}
Detailed (per-class) classification report:
{
    "0.0": {
        "precision": 0.7050793323717288,
        "recall": 0.9595485137408861,
        "f1-score": 0.812863760541632,
        "support": 14264
    },
    "1.0": {
        "precision": 0.7050102249488752,
        "recall": 0.194115990990991,
        "f1-score": 0.30441501103752766,
        "support": 7104
    },
    "accuracy": 0.7050730063646574,
    "macro avg": {
        "precision": 0.705044778660302,
        "recall": 0.5768322523659386,
        "f1-score": 0.5586393857895798,
        "support": 21368
    },
    "weighted avg": {
        "precision": 0.7050563569350033,
        "recall": 0.7050730063646574,
        "f1-score": 0.6438250

Predictions:   [0 1 1 ... 0 0 0]


In [35]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesClassifierEntr,0.739985,0.71,0.224589,0.105948,0.302738,0.224589,0.105948,0.302738,0,True,4
1,ExtraTreesClassifierGini,0.739798,0.71,0.226053,0.105911,0.345051,0.226053,0.105911,0.345051,0,True,3
2,LightGBMClassifierXT,0.739657,0.74,0.049918,0.003709,0.274369,0.049918,0.003709,0.274369,0,True,8
3,RandomForestClassifierGini,0.738628,0.70,0.114872,0.105667,0.337162,0.114872,0.105667,0.337162,0,True,1
4,RandomForestClassifierEntr,0.737973,0.69,0.123669,0.105312,0.327921,0.123669,0.105312,0.327921,0,True,2
5,CatboostClassifier,0.737364,0.72,0.008056,0.003978,0.915800,0.008056,0.003978,0.915800,0,True,9
6,NeuralNetClassifier,0.718925,0.72,0.524147,0.012079,5.260593,0.524147,0.012079,5.260593,0,True,10
7,LightGBMClassifierCustom,0.709940,0.73,0.015163,0.002543,0.354200,0.015163,0.002543,0.354200,0,True,11
8,weighted_ensemble_k0_l1,0.705073,0.77,0.024959,0.006476,0.883696,0.003812,0.000000,0.272763,1,True,12
9,LightGBMClassifier,0.704558,0.76,0.005984,0.003932,0.256732,0.005984,0.003932,0.256732,0,True,7


# AutoGluon Maxime accuracy

In [7]:
train_data = task.Dataset(train)
test_data = task.Dataset(test)
label_column="signal"
output_directory=current_path+ "\\trained_models"

In [9]:
time_limits = 10800 # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc' # specify your evaluation metric here
#presets="medium_quality_faster_train" # produces less accurate models but facilitates faster prototyping
presets='best_quality'
predictor = task.fit(train_data=train_data, label=label_column, time_limits=time_limits,
                     eval_metric=metric, presets=presets,output_directory=output_directory,verbosity=3)

Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to D:\Uni\2020.ml.kaggle\Marc\\trained_models\
AutoGluon Version:  0.0.14
Train Data Rows:    210563
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1.0, 0.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    36383.93 MB
	Train Data (Original)  Memory Usage: 25.27 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting A

Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\utils\model_template.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\model.pkl
	0.8397	 = Validation roc_auc score
	224.2s	 = Training runtime
	7.47s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Fitting model: KNeighborsClassifierUnif_STACKER_l0 ... Training model for up to 3916.36s of the 9316.04s of remaining time.
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\utils\model_template.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\model.pkl
	0.6706	 = Validation roc_auc score
	12.78s	 = Training ru

[50]	train_set's binary_logloss: 0.455314	valid_set's binary_logloss: 0.465341
[100]	train_set's binary_logloss: 0.43159	valid_set's binary_logloss: 0.447267
[150]	train_set's binary_logloss: 0.417711	valid_set's binary_logloss: 0.438787
[200]	train_set's binary_logloss: 0.407555	valid_set's binary_logloss: 0.433888
[250]	train_set's binary_logloss: 0.39952	valid_set's binary_logloss: 0.430532
[300]	train_set's binary_logloss: 0.392531	valid_set's binary_logloss: 0.428926
[350]	train_set's binary_logloss: 0.386249	valid_set's binary_logloss: 0.427696
[400]	train_set's binary_logloss: 0.380325	valid_set's binary_logloss: 0.426073
[450]	train_set's binary_logloss: 0.374807	valid_set's binary_logloss: 0.425446
[500]	train_set's binary_logloss: 0.369644	valid_set's binary_logloss: 0.424787
[550]	train_set's binary_logloss: 0.364613	valid_set's binary_logloss: 0.424292


Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True}


[50]	train_set's binary_logloss: 0.455447	valid_set's binary_logloss: 0.46335
[100]	train_set's binary_logloss: 0.431911	valid_set's binary_logloss: 0.445265
[150]	train_set's binary_logloss: 0.417795	valid_set's binary_logloss: 0.435608
[200]	train_set's binary_logloss: 0.407816	valid_set's binary_logloss: 0.430644
[250]	train_set's binary_logloss: 0.399666	valid_set's binary_logloss: 0.42739
[300]	train_set's binary_logloss: 0.392591	valid_set's binary_logloss: 0.425339
[350]	train_set's binary_logloss: 0.386344	valid_set's binary_logloss: 0.423971
[400]	train_set's binary_logloss: 0.380508	valid_set's binary_logloss: 0.42284
[450]	train_set's binary_logloss: 0.374849	valid_set's binary_logloss: 0.421889
[500]	train_set's binary_logloss: 0.369584	valid_set's binary_logloss: 0.421156
[550]	train_set's binary_logloss: 0.364475	valid_set's binary_logloss: 0.420207
[600]	train_set's binary_logloss: 0.359594	valid_set's binary_logloss: 0.419506
[650]	train_set's binary_logloss: 0.354917	v

...

Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'seed_value': 0}


[50]	train_set's binary_logloss: 0.368985	valid_set's binary_logloss: 0.377851
[100]	train_set's binary_logloss: 0.331868	valid_set's binary_logloss: 0.349435
[150]	train_set's binary_logloss: 0.318843	valid_set's binary_logloss: 0.346027


Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'seed_value': 0}


[50]	train_set's binary_logloss: 0.368656	valid_set's binary_logloss: 0.379477
[100]	train_set's binary_logloss: 0.331496	valid_set's binary_logloss: 0.351465
[150]	train_set's binary_logloss: 0.318466	valid_set's binary_logloss: 0.348143


Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierCustom_STACKER_l1\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierCustom_STACKER_l1\model.pkl
	0.9149	 = Validation roc_auc score
	29.87s	 = Training runtime
	0.69s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Not enough time left to finish repeated k-fold bagging, stopping early ...
Completed 1/20 k-fold bagging repeats ...
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierEntr_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsCl

In [10]:
predictor.leaderboard(test_data, silent=True)

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierDist_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierCustom_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\NeuralNetClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClass

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMClassifier_STACKER_l1,0.925232,0.915023,167.003345,45.026828,5279.473441,0.077793,0.216031,11.059959,1,True,19
1,LightGBMClassifierCustom_STACKER_l1,0.925135,0.914871,167.254024,45.498754,5298.280874,0.328472,0.687957,29.867392,1,True,23
2,NeuralNetClassifier_STACKER_l0,0.925121,0.914955,0.606634,5.186029,3149.454976,0.606634,5.186029,3149.454976,0,True,10
3,LightGBMClassifierXT_STACKER_l1,0.925032,0.914932,167.038249,45.199545,5283.664114,0.112698,0.388748,15.250632,1,True,20
4,weighted_ensemble_k0_l2,0.924991,0.915329,215.839642,60.590777,9138.759186,0.004956,0.065884,74.335022,2,True,24
5,CatboostClassifier_STACKER_l1,0.924797,0.915090,167.000025,44.880546,5306.636448,0.074473,0.069749,38.222966,1,True,21
6,weighted_ensemble_k0_l1,0.924710,0.915017,0.722723,5.401381,3696.028954,0.010849,0.072114,73.914649,1,True,12
7,NeuralNetClassifier_STACKER_l1,0.924246,0.914300,167.706728,49.621700,7906.473297,0.781176,4.810903,2638.059815,1,True,22
8,RandomForestClassifierEntr_STACKER_l1,0.923204,0.913356,178.877064,48.895810,6027.321072,11.951513,4.085013,758.907590,1,True,14
9,ExtraTreesClassifierEntr_STACKER_l1,0.922603,0.913250,202.508561,50.266493,5573.055811,35.583009,5.455696,304.642329,1,True,16


In [15]:
y_pred = predictor.predict(test_data)
y_test=test_data["signal"]
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: roc_auc on test data: 0.8295465660588275
Evaluations on test data:
{
    "roc_auc": 0.8295465660588275,
    "accuracy_score": 0.8561219628394473,
    "balanced_accuracy_score": 0.8295465660588276,
    "matthews_corrcoef": 0.6780723832316425,
    "f1_score": 0.8561219628394473
}
Detailed (per-class) classification report:
{
    "0.0": {
        "precision": 0.8683298683298684,
        "recall": 0.9179487179487179,
        "f1-score": 0.8924501424501424,
        "support": 1365
    },
    "1.0": {
        "precision": 0.8292682926829268,
        "recall": 0.7411444141689373,
        "f1-score": 0.7827338129496403,
        "support": 734
    },
    "accuracy": 0.8561219628394473,
    "macro avg": {
        "precision": 0.8487990805063976,
        "recall": 0.8295465660588276,
        "f1-score": 0.8375919776998914,
        "support": 2099
    },
    "weighted avg": {
        "precision": 0.854670413101257,
        "recall": 0.8561219628394473,
        "f1-score": 0.85408340312

Predictions:   [0 0 1 ... 0 0 0]


## Create csv

In [11]:
# We only get the train data set
current_path = os.getcwd()+"\\"
file = os.path.sep.join(["..\\data\\test.csv"])
data = pd.read_csv(current_path + file)
data=data.set_index("Id")

# signal is the target variable
df_test=data.drop(["BUTTER"], axis=1) # Droping BUTTER variable as is IRRELEVANT

In [12]:
y_pred = predictor.predict_proba(df_test)
y_pred

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_k0_l2\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierDist_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierCustom_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\NeuralNetClassifier_STAC

array([0.05226269, 0.04656867, 0.088664  , ..., 0.64286876, 0.6214867 ,
       0.02711915], dtype=float32)

In [13]:
results=pd.DataFrame(columns=["Predicted"],data=y_pred)
results.to_csv("WinResults2.csv")